<a href="https://colab.research.google.com/github/Manmithalder/Plot-Based-Movie-Recommendation-System/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd


In [ ]:
movies_data = pd.read_csv('/content/movies.csv')
movies_data.head()

In [ ]:
selected_features = ['genres','keywords','tagline','cast','director','overview','crew']

In [ ]:
for feature in selected_features:
  movies_data[feature] = movies_data[feature].fillna('')

In [ ]:
movie_description = input('enter movie description: ')

enter movie description: rgbdrj


In [ ]:
list_of_all_overviews = movies_data['overview'].tolist()
print(list_of_all_overviews)

In [ ]:
!pip install -q sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.6 MB/s eta 0:00:00


In [ ]:
!pip install -q sentence-transformers faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 55.3 MB/s eta 0:00:00


In [ ]:
import torch
import faiss
from sentence_transformers import SentenceTransformer, util
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = SentenceTransformer('all-MiniLM-L6-v2').to(device)

# Encode all movie overviews **once** (Precompute embeddings)
batch_size = 512  # Optimize memory usage
sentence_embeddings = model.encode(list_of_all_overviews, batch_size=batch_size, convert_to_numpy=True)

# Normalize for cosine similarity
sentence_embeddings = sentence_embeddings / np.linalg.norm(sentence_embeddings, axis=1, keepdims=True)

# Create FAISS index for fast search
embedding_dim = sentence_embeddings.shape[1]  # Get embedding size (384 for MiniLM)
index = faiss.IndexFlatIP(embedding_dim)  # Inner Product = Cosine Similarity
index.add(sentence_embeddings)  # Store embeddings in FAISS

def bert_get_closest_sentence(input_sentence, n=1):
    """
    Finds the closest matching sentence(s) using FAISS.
    """
    # Encode the input sentence **only once**
    input_embedding = model.encode([input_sentence], convert_to_numpy=True)
    input_embedding = input_embedding / np.linalg.norm(input_embedding)  # Normalize

    # Use FAISS for fast similarity search
    D, I = index.search(input_embedding, k=n)  # Get top-n closest matches

    return [list_of_all_overviews[idx] for idx in I[0]]  # Return closest matches


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
movie_description = input('enter movie description: ')

closest_match = bert_get_closest_sentence(movie_description, n=10)
close_match = closest_match[0:10]

title_of_the_movie = [movies_data[movies_data['overview'] == overview]['title'].values[0] for overview in close_match]

for title, overview in zip(title_of_the_movie, close_match):
    print(f"\n  {title}: {overview}")
